## Ciabattini

- bulk fermentation: 3hrs, fold twice
- divide & shape
- final fermentation: 1.5 hrs


In [53]:
import pandas as pd
grams_per_ounce = 28.3495

In [54]:
formatter = {
    'grams': lambda x: f"{x:,.0f}" if x > 10 else f"{x:.2f}",
    'oz': lambda x: f"{x:,.0f}" if x > 100 else f"{x:.2f}",
    'baker%': lambda x: f"{x:.0f}" if x > 2 else f"{x:.1f}"
}

In [55]:
num_loaves = 45
num_pounds = 0
num_ounces = 2.5
num_grams = 0

In [56]:
loaf_weight = (num_pounds * 16 + num_ounces) * grams_per_ounce + num_grams
total_weight = (num_loaves * loaf_weight)
print ("{:,.0f} loaves at {:,.0f} grams".
       format(num_loaves, loaf_weight, num_pounds, num_ounces))
print ("total weight: {:,.2f} grams".format(total_weight,total_weight * grams_per_ounce))

45 loaves at 71 grams
total weight: 3,189.32 grams


In [57]:
pre_fermented_flour = .30
formula = pd.DataFrame.from_dict({
    "flour": 100, 
    "water": 73, 
    "yeast": .4,
    "salt": 2,
}, orient='index', columns=["baker%"])
formula_total = formula['baker%'].sum()
print("overall formula total = %s%%" % formula_total)

overall formula total = 175.4%


In [58]:
flour_weight = formula.at['flour', 'baker%'] / formula_total * total_weight
formula['grams'] = formula.apply(lambda row: row['baker%'] * flour_weight / 100, axis=1)
formula['oz'] = formula.apply(lambda row: row['grams'] / grams_per_ounce, axis=1)
print ("overall formula")

formula[['baker%','grams','oz']].style.format(formatter)

overall formula


,baker%,grams,oz
flour,100,"1,818",64.14
water,73,"1,327",46.82
yeast,0.4,7.27,0.26
salt,2.0,36,1.28


In [59]:
poolish = pd.DataFrame.from_dict({
    "flour": 100,
    "water": 100, 
    "yeast": 0.07
}, orient='index', columns=["baker%"])
print ("poolish")
poolish_total = poolish['baker%'].sum()
print("poolish total = %s%%" % poolish_total)
poolish.style.format(formatter)

poolish
poolish total = 200.07%


,baker%
flour,100
water,100
yeast,0.1


In [60]:
poolish_flour = total_weight / formula_total * formula.at['flour','baker%'] * pre_fermented_flour
poolish_weight = poolish_flour * poolish_total / poolish.at['flour','baker%']
print ("poolish weight = {:,.0f} grams".format(poolish_weight))
poolish['grams'] = poolish.apply(lambda row: row['baker%']*poolish_weight/poolish_total, axis=1)
poolish['oz'] = poolish.apply(lambda row: float(row['grams']) / grams_per_ounce, axis=1)

poolish[['baker%', 'grams', 'oz']].style.format(formatter)

poolish weight = 1,091 grams


,baker%,grams,oz
flour,100,545,19.24
water,100,545,19.24
yeast,0.1,0.38,0.01


In [61]:
final_dough = formula.copy()
final_dough.at['poolish','grams'] = poolish_weight
final_dough.at['poolish','baker%'] = poolish_weight / flour_weight * 100
for index, row in poolish.iterrows():
    final_dough.at[index,'grams'] = final_dough.at[index,'grams'] - row['grams']
final_dough['oz'] = final_dough.apply(lambda row: row['grams']/grams_per_ounce, axis=1)
print("final dough")
final_dough[['baker%', 'grams', 'oz']].style.format(formatter)

final dough


,baker%,grams,oz
flour,100,"1,273",44.90
water,73,782,27.58
yeast,0.4,6.89,0.24
salt,2.0,36,1.28
poolish,60,"1,091",38.50


In [62]:
display_df = final_dough.copy()
for col, fmt in formatter.items():
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(fmt)
print(display_df.to_markdown())

|         |   baker% | grams   |    oz |
|:--------|---------:|:--------|------:|
| flour   |    100   | 1,273   | 44.9  |
| water   |     73   | 782     | 27.58 |
| yeast   |      0.4 | 6.89    |  0.24 |
| salt    |      2   | 36      |  1.28 |
| poolish |     60   | 1,091   | 38.5  |
